In [1]:
import plotly.graph_objects as go
import pandas as pd

In [2]:
# Load the data
data = pd.read_csv('./generalAccuracy_combined.csv.bz2', compression='bz2')

# Overall Accuracy

In [14]:
sampling_method = 'RankSampling' # choose a sampling strategy: 'RandomSampling', 'OversampleMinority', or 'RankSampling'
bias_applied = True # data with or without the presence of bias


filtered_df = data.loc[(data.bias_applied==bias_applied) & (data.metric=='tau')
                       & (data.group=='Overall') & (data['sampling method']==sampling_method)].copy()
filtered_df['Weighted Kemeny Distance'] = data.value
grouped = filtered_df.groupby(by=['iteration', 'bias_applied', 'sampling method', 'metric', 'group', 'Ranking Method'])
mean_df = grouped.mean().reset_index()

In [15]:
fig = go.Figure()

methods = ['randomRankRecovery', 'btl', 'davidScore', 'eigenvectorCentrality', 'rankCentrality',
           'PageRank', 'fairPageRank', 'SerialRank', 'GNNRank', 'SVD_RS', 'SVD_NRS']

for ranking_method in methods:
    y = mean_df.loc[(mean_df['Ranking Method'] == ranking_method), 'value']
    x = mean_df.loc[(mean_df['Ranking Method'] == ranking_method), 'iteration']
    if ranking_method not in ['rankCentrality', 'GNNRank', 'fairPageRank', 'randomRankRecovery']:
        visible = "legendonly"
    else: visible=True
    fig.add_trace(go.Scatter(x=x, y=y, name=ranking_method, visible=visible)) # the line for the mean
    fig.add_trace(go.Scatter()) # the error bar

fig.update_layout(
    yaxis_title='Weighted Kemeny Distance',
    xaxis_title='iteration',
    title=f'Overall Accuracy for the {sampling_method} strategy, with{"out" if not bias_applied else ""} bias',
    yaxis_range=[0,1],
    xaxis_range=[0,3000],
    height=600,
    width=1000
)

fig.show()
